In [1]:
#!pip install spacy
#!python -m spacy download en_core_web_sm
#!pip install -U scikit-learn
#!pip install summa

In [2]:
import pandas as pd
import numpy as np
import warnings
import json
import string
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

Getting feedback data from amazon Giftcards and amazon All beauty

In [3]:
def lowerpunct(data_frame):                              #function to lowercase all alphabets and remove punctuations
     data_frame['reviewText'] = data_frame['reviewText'].apply(lambda x: ''.join([char.lower()
                                                                                     if not pd.isna(char)
                                                                                     and char not in string.punctuation
                                                                                     else ' ' for char in str(x)]))

In [4]:
file1="AMAZON_FASHION.json"
ans1=[]
with open(file1,'r') as json_file:
    for line in json_file:
        data=json.loads(line)
        if "overall" in data and "reviewText" in data:
            ans1.append([data["overall"],data["reviewText"]])
            
a1 = pd.DataFrame(ans1)
a1.rename(columns = {0:"overall", 1:"reviewText"}, inplace = True)
a1.head()

,overall,reviewText
0,5.0,Exactly what I needed.
1,2.0,"I agree with the other review, the opening is ..."
2,4.0,Love these... I am going to order another pack...
3,2.0,too tiny an opening
4,3.0,Okay


In [5]:
file2 = "Patio_Lawn_and_Garden.json"
nlp = spacy.load("en_core_web_sm")
ans2=[]
with open(file2,'r') as json_file:
    for line in json_file:
        data=json.loads(line)
        if "overall" in data and "reviewText" in data:
            ans2.append([data["overall"],data["reviewText"]])
a2 = pd.DataFrame(ans2)
a2.rename(columns = {0:"overall", 1:"reviewText"}, inplace = True)
a2.tail()

,overall,reviewText
5233378,5.0,Very nice good quality I am very happy with it!
5233379,1.0,Dont buy this umbrella!!! In two weeks the br...
5233380,4.0,Pretty and the right price
5233381,5.0,Just what we needed for our Montgomery Ward's ...
5233382,5.0,Customer had the part number....so just what h...


In [6]:
warnings.filterwarnings('ignore')

In [7]:
df = a1[:10000].append(a2[-10000:], ignore_index=True)
                                  # merging 2 datsets from Amazon Gift Cards and All Beauty first and last rows each
print(df.head(),"\n")
print(df.tail(),"\n")

   overall                                         reviewText
0      5.0                             Exactly what I needed.
1      2.0  I agree with the other review, the opening is ...
2      4.0  Love these... I am going to order another pack...
3      2.0                                too tiny an opening
4      3.0                                               Okay 

       overall                                         reviewText
19995      5.0    Very nice good quality I am very happy with it!
19996      1.0  Dont buy this umbrella!!!  In two weeks the br...
19997      4.0                         Pretty and the right price
19998      5.0  Just what we needed for our Montgomery Ward's ...
19999      5.0  Customer had the part number....so just what h... 



In [8]:
# Converting the reviews into lowercase and removing punctuation
lowerpunct(df)
df.head()

,overall,reviewText
0,5.0,exactly what i needed
1,2.0,i agree with the other review the opening is ...
2,4.0,love these i am going to order another pack...
3,2.0,too tiny an opening
4,3.0,okay


In [9]:
tfidfVectorizer = TfidfVectorizer()                          # Calculating TF-IDF
tfidf_matrix = tfidfVectorizer.fit_transform(df["reviewText"])
feature_names = tfidfVectorizer.get_feature_names_out()
idf_df = pd.DataFrame({'word':feature_names,'idf':tfidfVectorizer.idf_})
idf_df_sorted = idf_df.sort_values(by='idf')

print("Top 30 words based on IDF:")
print(idf_df_sorted.head(30))
print("Bottom 30 words based on IDF:")
print(idf_df_sorted.tail(30))

Top 30 words based on IDF:
        word       idf
14151    the  1.738299
1215     and  1.790590
14366     to  2.068473
7732      it  2.082100
5995     for  2.191436
7721      is  2.410842
9293      my  2.421972
9644      of  2.483737
7404      in  2.497947
14227   this  2.512588
2588     but  2.512815
6569   great  2.606991
15347    was  2.697500
9511     not  2.705173
14190   they  2.745021
9688      on  2.779683
15169   very  2.792510
15678   with  2.812523
14147   that  2.870853
1375     are  2.875735
5789     fit  2.885900
14186  these  2.903523
6852    have  2.918734
1443      as  2.961886
12974     so  2.999572
6470    good  3.014453
14161   them  3.099286
1876      be  3.204601
8306    like  3.260055
7890    just  3.273562
Bottom 30 words based on IDF:
           word       idf
7930     kerrii  10.21039
7931      keter  10.21039
725          93  10.21039
7934   keyboard  10.21039
7935      keyed  10.21039
7937     keyway  10.21039
7938   keywords  10.21039
7942      khmer  10.21

In [10]:
nlp = spacy.load("en_core_web_sm")
df2 = a1[:10].append(a2[:10], ignore_index=True) # Combining dataset for first 10 reviews from each category
lowerpunct(df2)
for i, review_text in enumerate(df2['reviewText']):
            # Process the review text with spaCy
            doc = nlp(review_text)

            # Extract sentences and print review ID along with each sentence
            sentence_id = 0
            for sent in doc.sents:
                sentence_id += 1
                print(f"Review ID: {i}, Sentence {sentence_id}: {sent.text} \n")

Review ID: 0, Sentence 1: exactly what i needed 

Review ID: 1, Sentence 1: i agree with the other review  the opening is too small   i almost bent the hook on some very expensive earrings trying to get these up higher than just the end so they re not seen   would not buy again but for the price  not sending back 

Review ID: 2, Sentence 1: love these    i am going to order another pack to keep in work  someone  including myself  is always losing the back to an earring   i don t understand why all fish hook earrings don t have them   just wish that they were a tiny bit longer      

Review ID: 3, Sentence 1: too tiny an opening 

Review ID: 4, Sentence 1: okay 

Review ID: 5, Sentence 1: exactly what i wanted 

Review ID: 6, Sentence 1: these little plastic backs work great   no more loosing  hook  earrings   wish i had ordered these sooner before i had lost some of my favorite earrings 

Review ID: 7, Sentence 1: mother   in   law wanted it as a present for her sister  she liked it an

Now,
We will perform word tokenization

In [11]:
for index, review_text in enumerate(df2['reviewText']):
            # Process the review text with spaCy
            doc = nlp(review_text)

            # Extract token, lemma, and POS tag, and print review ID along with each token
            for token in doc:
                print(f"Review ID: {index}, Token: {token.text}, Lemma: {token.lemma_}, POS: {token.pos_}")


Review ID: 0, Token: exactly, Lemma: exactly, POS: ADV
Review ID: 0, Token: what, Lemma: what, POS: PRON
Review ID: 0, Token: i, Lemma: I, POS: PRON
Review ID: 0, Token: needed, Lemma: need, POS: VERB
Review ID: 1, Token: i, Lemma: I, POS: PRON
Review ID: 1, Token: agree, Lemma: agree, POS: VERB
Review ID: 1, Token: with, Lemma: with, POS: ADP
Review ID: 1, Token: the, Lemma: the, POS: DET
Review ID: 1, Token: other, Lemma: other, POS: ADJ
Review ID: 1, Token: review, Lemma: review, POS: NOUN
Review ID: 1, Token:  , Lemma:  , POS: SPACE
Review ID: 1, Token: the, Lemma: the, POS: DET
Review ID: 1, Token: opening, Lemma: opening, POS: NOUN
Review ID: 1, Token: is, Lemma: be, POS: AUX
Review ID: 1, Token: too, Lemma: too, POS: ADV
Review ID: 1, Token: small, Lemma: small, POS: ADJ
Review ID: 1, Token:   , Lemma:   , POS: SPACE
Review ID: 1, Token: i, Lemma: I, POS: PRON
Review ID: 1, Token: almost, Lemma: almost, POS: ADV
Review ID: 1, Token: bent, Lemma: bend, POS: VERB
Review ID: 1, Tok

Review ID: 8, Token: item, Lemma: item, POS: NOUN
Review ID: 8, Token: is, Lemma: be, POS: AUX
Review ID: 8, Token: of, Lemma: of, POS: ADP
Review ID: 8, Token: good, Lemma: good, POS: ADJ
Review ID: 8, Token: quality, Lemma: quality, POS: NOUN
Review ID: 8, Token:  , Lemma:  , POS: SPACE
Review ID: 8, Token: looks, Lemma: look, POS: VERB
Review ID: 8, Token: great, Lemma: great, POS: ADJ
Review ID: 8, Token:  , Lemma:  , POS: SPACE
Review ID: 8, Token: too, Lemma: too, POS: ADV
Review ID: 8, Token:  , Lemma:  , POS: SPACE
Review ID: 8, Token: but, Lemma: but, POS: CCONJ
Review ID: 8, Token: it, Lemma: it, POS: PRON
Review ID: 8, Token: does, Lemma: do, POS: AUX
Review ID: 8, Token: not, Lemma: not, POS: PART
Review ID: 8, Token: fit, Lemma: fit, POS: VERB
Review ID: 8, Token: 100, Lemma: 100, POS: NUM
Review ID: 8, Token: s, Lemma: s, POS: NOUN
Review ID: 8, Token:  , Lemma:  , POS: SPACE
Review ID: 8, Token: but, Lemma: but, POS: CCONJ
Review ID: 8, Token: it, Lemma: it, POS: PRON
Re

Review ID: 12, Token: i, Lemma: I, POS: PRON
Review ID: 12, Token: was, Lemma: be, POS: AUX
Review ID: 12, Token: having, Lemma: have, POS: VERB
Review ID: 12, Token: trouble, Lemma: trouble, POS: NOUN
Review ID: 12, Token: trying, Lemma: try, POS: VERB
Review ID: 12, Token: to, Lemma: to, POS: PART
Review ID: 12, Token: put, Lemma: put, POS: VERB
Review ID: 12, Token: into, Lemma: into, POS: ADP
Review ID: 12, Token: words, Lemma: word, POS: NOUN
Review ID: 12, Token: how, Lemma: how, POS: SCONJ
Review ID: 12, Token: i, Lemma: I, POS: PRON
Review ID: 12, Token: felt, Lemma: feel, POS: VERB
Review ID: 12, Token: about, Lemma: about, POS: ADP
Review ID: 12, Token: reading, Lemma: read, POS: VERB
Review ID: 12, Token: this, Lemma: this, POS: DET
Review ID: 12, Token: book, Lemma: book, POS: NOUN
Review ID: 12, Token:   , Lemma:   , POS: SPACE
Review ID: 12, Token: i, Lemma: I, POS: PRON
Review ID: 12, Token: read, Lemma: read, POS: VERB
Review ID: 12, Token: through, Lemma: through, POS:

Review ID: 16, Token: this, Lemma: this, POS: PRON
Review ID: 16, Token: is, Lemma: be, POS: AUX
Review ID: 16, Token: a, Lemma: a, POS: DET
Review ID: 16, Token: great, Lemma: great, POS: ADJ
Review ID: 16, Token: story, Lemma: story, POS: NOUN
Review ID: 16, Token: that, Lemma: that, POS: PRON
Review ID: 16, Token: had, Lemma: have, POS: VERB
Review ID: 16, Token: me, Lemma: I, POS: PRON
Review ID: 16, Token: truly, Lemma: truly, POS: ADV
Review ID: 16, Token: hooked, Lemma: hook, POS: VERB
Review ID: 16, Token: from, Lemma: from, POS: ADP
Review ID: 16, Token: the, Lemma: the, POS: DET
Review ID: 16, Token: beginning, Lemma: beginning, POS: NOUN
Review ID: 16, Token:  , Lemma:  , POS: SPACE
Review ID: 16, Token: i, Lemma: I, POS: PRON
Review ID: 16, Token: really, Lemma: really, POS: ADV
Review ID: 16, Token: enjoy, Lemma: enjoy, POS: VERB
Review ID: 16, Token: a, Lemma: a, POS: DET
Review ID: 16, Token: murder, Lemma: murder, POS: NOUN
Review ID: 16, Token: mystery, Lemma: mystery,

Review ID: 19, Token: the, Lemma: the, POS: DET
Review ID: 19, Token: premise, Lemma: premise, POS: NOUN
Review ID: 19, Token: sounded, Lemma: sound, POS: VERB
Review ID: 19, Token: interesting, Lemma: interesting, POS: ADJ
Review ID: 19, Token: and, Lemma: and, POS: CCONJ
Review ID: 19, Token: this, Lemma: this, POS: PRON
Review ID: 19, Token: could, Lemma: could, POS: AUX
Review ID: 19, Token: have, Lemma: have, POS: AUX
Review ID: 19, Token: been, Lemma: be, POS: AUX
Review ID: 19, Token: a, Lemma: a, POS: DET
Review ID: 19, Token: good, Lemma: good, POS: ADJ
Review ID: 19, Token: mystery, Lemma: mystery, POS: NOUN
Review ID: 19, Token:   , Lemma:   , POS: SPACE
Review ID: 19, Token: the, Lemma: the, POS: DET
Review ID: 19, Token: writing, Lemma: writing, POS: NOUN
Review ID: 19, Token: isn, Lemma: isn, POS: ADP
Review ID: 19, Token: t, Lemma: t, POS: NOUN
Review ID: 19, Token: bad, Lemma: bad, POS: ADJ
Review ID: 19, Token: and, Lemma: and, POS: CCONJ
Review ID: 19, Token: the, Lem

Dataset Preparation Classification is shown below

In [12]:
from sklearn.model_selection import train_test_split

# Load and prepare the data for Training
Train_Data_1 = a1.iloc[0:int(len(a1)*0.80)]
lowerpunct(Train_Data_1)
Train_Data_2 = a2.iloc[0:int(len(a2)*0.80)]
lowerpunct(Train_Data_2)
Train_Dataset = Train_Data_1.append(Train_Data_2)
Train_Dataset.head()

,overall,reviewText
0,5.0,exactly what i needed
1,2.0,i agree with the other review the opening is ...
2,4.0,love these i am going to order another pack...
3,2.0,too tiny an opening
4,3.0,okay


In [13]:
# Load and prepare the data for Testing
Test_Data_1 = a1.iloc[int(len(a1)*(-0.20)):]
lowerpunct(Test_Data_1)
Test_Data_2 = a2.iloc[int(len(a2)*(-0.20)):]
lowerpunct(Test_Data_2)
Test_Dataset = Test_Data_1.append(Test_Data_2)
Test_Dataset.head()

,overall,reviewText
705923,5.0,as exspected
705924,1.0,material is very cheap and thin it was to smal...
705925,2.0,horrible material
705926,2.0,not the color i thought it would be
705927,5.0,amazing pants i loved them


In [14]:
Train_Dataset['reviewText'] = Train_Dataset['reviewText'].fillna('').astype(str)
Test_Dataset['reviewText'] = Test_Dataset['reviewText'].fillna('').astype(str)

Binary Classification starts from below

In [15]:
# Train Data for Binary classification problem 
Train_Dataset['binary_class'] = np.where(Train_Dataset['overall'] == 5, 1, 0)  
                                                             #rating 5 is assigned 1 and remaining ratings as 0
class_0_Train = Train_Dataset[Train_Dataset['binary_class'] == 0]  #divide 0 and 1 classes 
class_1_Train = Train_Dataset[Train_Dataset['binary_class'] == 1]
sample_size = min(len(class_0_Train), len(class_1_Train))
class_0_sample = class_0_Train.sample(n=sample_size, random_state=42)
class_1_sample = class_1_Train.sample(n=sample_size, random_state=42)
Train_data_binary_balanced = pd.concat([class_0_sample, class_1_sample])

In [16]:
# Test Data for Binary classification problem 
Test_Dataset['binary_class'] = np.where(Test_Dataset['overall'] == 5, 1, 0)  
                                                             #rating 5 is assigned 1 and remaining ratings as 0
class_0_Test = Test_Dataset[Test_Dataset['binary_class'] == 0]  #divide 0 and 1 classes 
class_1_Test = Test_Dataset[Test_Dataset['binary_class'] == 1]
sample_size_t = min(len(class_0_Test), len(class_1_Test))
class_0_sample_t = class_0_Test.sample(n=sample_size_t, random_state=42)
class_1_sample_t = class_1_Test.sample(n=sample_size_t, random_state=42)
Test_data_binary_balanced = pd.concat([class_0_sample_t, class_1_sample_t])

In [17]:
# Splitting for binary classification
X_train_bin = Train_data_binary_balanced["reviewText"]
y_train_bin = Train_data_binary_balanced["binary_class"]
X_test_bin = Test_data_binary_balanced["reviewText"]
y_test_bin = Test_data_binary_balanced["binary_class"]

In [18]:
# TFIDF Vectorizer
tfidf_vectorizer_bin = TfidfVectorizer(max_features=50000)

In [19]:
# Vectorization for both classification types
X_train_tfidf_bin = tfidf_vectorizer_bin.fit_transform(X_train_bin)
X_test_tfidf_bin = tfidf_vectorizer_bin.transform(X_test_bin)

In [20]:
from sklearn.naive_bayes import MultinomialNB

# Multinomial Naive Bayes Model
model_bin = MultinomialNB()
# Train the model
model_bin.fit(X_train_tfidf_bin, y_train_bin)

MultinomialNB()

In [21]:
# Predictions
y_pred_bin = model_bin.predict(X_test_tfidf_bin)
print(y_pred_bin)

[0 0 0 ... 1 0 1]


In [22]:
# Evaluation of Accuracy of Predicted testing data

from sklearn.metrics import classification_report
accuracy_bin = classification_report(y_test_bin, y_pred_bin, zero_division=0)
print("Binary Classification Report:\n", accuracy_bin)

Binary Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.81      0.82    499893
           1       0.81      0.82      0.82    499893

    accuracy                           0.82    999786
   macro avg       0.82      0.82      0.82    999786
weighted avg       0.82      0.82      0.82    999786



Multiclass Classification starts from below

In [23]:
#Train and Test dataset of Multi-class classification
X_train_5 = Train_Dataset["reviewText"]
y_train_5 = Train_Dataset["overall"]
X_test_5 = Test_Dataset["reviewText"]
y_test_5 = Test_Dataset["overall"]

In [24]:
# TFIDF Vectorizer
tfidf_vectorizer_5 = TfidfVectorizer(max_features=50000)

In [25]:
# Vectorization for 5 classification types
X_train_tfidf_5 = tfidf_vectorizer_5.fit_transform(X_train_5)
X_test_tfidf_5 = tfidf_vectorizer_5.transform(X_test_5)

In [26]:
# Multinomial Naive Bayes Model
model_5 = MultinomialNB()
# Train the model
model_5.fit(X_train_tfidf_5, y_train_5)

MultinomialNB()

In [27]:
y_pred_5 = model_5.predict(X_test_tfidf_5)
print(y_pred_5)

[5. 5. 1. ... 5. 5. 5.]


In [28]:
# Evaluation of Accuracy of Predicted testing data

accuracy_5 = classification_report(y_test_5, y_pred_5, zero_division=0)
print("Multi-Class Classification Report:\n", accuracy_5)

Multi-Class Classification Report:
               precision    recall  f1-score   support

         1.0       0.73      0.48      0.58    172186
         2.0       0.29      0.00      0.00     73933
         3.0       0.35      0.06      0.10     97294
         4.0       0.41      0.06      0.10    156480
         5.0       0.67      0.99      0.80    723263

    accuracy                           0.66   1223156
   macro avg       0.49      0.32      0.32   1223156
weighted avg       0.60      0.66      0.57   1223156



Text Summarization is shown below 

In [ ]:
from summa import summarizer
reviews_5 = a1[a1['overall'] == 5.0][:1000].append(a2[a2['overall'] == 5.0][:1000])
print(reviews_5)

review_5_rating = ''
for i in range(len(reviews_5['reviewText'])):
    review_5_rating = review_5_rating+ ' ' +str(reviews_5['reviewText'].iloc[i]) 
summary_5 = summarizer.summarize(review_5_rating, words=300)

print("\nSummary for Rating 5.0 reviews (300 words):")
print(summary_5)


rating_1_reviews = a1[a1['overall'] == 1.0][:1000].append(a2[a2['overall'] == 1.0][:1000])
print(rating_1_reviews)

review_1_rating = ''
for i in range(len(rating_1_reviews['reviewText'])):
    review_1_rating = review_1_rating+ ' ' +str(rating_1_reviews['reviewText'].iloc[i]) 
    
summary_1 = summarizer.summarize(review_1_rating, ratio=0.01)

print("Summarized 'rating-1.0' reviews:")
print(summary_1)